In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!pip install pip install tensorflow-addons


Mounted at /content/gdrive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.1 MB/s eta 0:00:00


In [2]:
import keras 
import pandas as pd
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from IPython import display
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Layer, Conv2D, Flatten, Dense, Reshape, Conv2DTranspose
from tensorflow.keras import Input, Model
from tensorflow.keras.utils import img_to_array
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import os
import random
import math
import numpy as np
import matplotlib.pyplot as plt

from enum import Enum
from glob import glob
from functools import partial

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from PIL import Image


import gdown
from zipfile import ZipFile
# for reproducibility - ref https://machinelearningmastery.com/reproducible-results-neural-networks-keras/ and https://www.tensorflow.org/api_docs/python/tf/keras/utils/set_random_seed
np.random.seed(9)
tf.keras.utils.set_random_seed(10)

# loading data from gdrive
covid19_dataset = os.path.abspath("/content/gdrive/My Drive/COVID-19 Dataset/")

!ls "/content/gdrive/My Drive/COVID-19 Dataset/"


CT  X-ray


In [3]:
#@markdown #**Anti-Disconnect for Google Colab**
#@markdown ## Run this to stop it from disconnecting automatically 
#@markdown  **(It will anyhow disconnect after 6 - 12 hrs for using the free version of Colab.)**
#@markdown  *(Colab Pro users will get about 24 hrs usage time)*
#@markdown ---
# taken from https://colab.research.google.com/github/justinjohn0306/VQGAN-CLIP/blob/main/VQGAN%2BCLIP_%28z%2Bquantize_method_with_augmentations%2C_user_friendly_interface%29.ipynb#scrollTo=XHyPd4oxVp_l stops colab disconnecting
import IPython
js_code = '''
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
'''
IPython.display.Javascript(js_code)


<IPython.core.display.Javascript object>

# Xray COVID

In [ ]:
# load images
image_size = (128,128)
img_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    covid19_dataset + '/X-ray/COVID/', label_mode=None, image_size=image_size, batch_size=16,crop_to_aspect_ratio=True
)

dataset = img_dataset.map(lambda x: x / 255.0)


Found 4044 files belonging to 1 classes.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
num_channels = 3
num_classes = 1
latent_dim = 256

In [ ]:
generator_in_channels = latent_dim + num_classes
discriminator_in_channels = num_channels + num_classes
print(generator_in_channels, discriminator_in_channels)

257 4


In [ ]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(128, 128, 3)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.5),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.5),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.5),
        layers.Flatten(),
        layers.Dropout(0.4),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)
discriminator.summary()

# Create the generator.
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(16 * 16 * 128),
        layers.Reshape((16, 16, 128)),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(1024, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, kernel_size=4, padding="same", activation="tanh"),
    ],
    name="generator",
)
generator.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 64, 64, 64)        3136      
                                                                 
 leaky_re_lu_22 (LeakyReLU)  (None, 64, 64, 64)        0         
                                                                 
 conv2d_15 (Conv2D)          (None, 32, 32, 128)       131200    
                                                                 
 leaky_re_lu_23 (LeakyReLU)  (None, 32, 32, 128)       0         
                                                                 
 conv2d_16 (Conv2D)          (None, 16, 16, 128)       262272    
                                                                 
 leaky_re_lu_24 (LeakyReLU)  (None, 16, 16, 128)       0         
                                                                 
 flatten_3 (Flatten)         (None, 32768)           

In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }
    def get_gan():
      return GAN(name='DC_GAN_COVID19_COVID_Xray')



In [ ]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=latent_dim):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        imageFolder = 0
        for i in range(self.num_img):
            img = tf.keras.preprocessing.image.array_to_img(generated_images[i])
            img.save('/content/gdrive/My Drive/COVID-19 Data Augmented COVID X-Ray' + '/' + "generated_img_%03d_%d.png" % (epoch, i))


In [ ]:
epochs = 200  # In practice, use ~100 epochs 


gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.RMSprop(learning_rate=0.00001,momentum=0),
    g_optimizer=keras.optimizers.RMSprop(learning_rate=0.00001,momentum=0),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

history = gan.fit(
    dataset, epochs=epochs, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)]
)


model = gan.get_gan
# Save the model
generator.save('/content/gdrive/My Drive/COVID-19_Augmented_COVID_XRayModel/Generator',save_format='tf')
discriminator.save('/content/gdrive/My Drive/COVID-19_Augmented_COVID_XRayModel/Discriminator',save_format='tf')


Epoch 1/200
253/253 [==============================] - 31s 109ms/step - d_loss: 0.6906 - g_loss: 0.7612
Epoch 2/200
253/253 [==============================] - 28s 110ms/step - d_loss: 0.6922 - g_loss: 0.7454
Epoch 3/200
253/253 [==============================] - 28s 110ms/step - d_loss: 0.6912 - g_loss: 0.7542
Epoch 4/200
253/253 [==============================] - 28s 110ms/step - d_loss: 0.6895 - g_loss: 0.7526
Epoch 5/200
253/253 [==============================] - 28s 110ms/step - d_loss: 0.6872 - g_loss: 0.7603
Epoch 6/200
253/253 [==============================] - 28s 110ms/step - d_loss: 0.6870 - g_loss: 0.7561
Epoch 7/200
253/253 [==============================] - 28s 109ms/step - d_loss: 0.6883 - g_loss: 0.7535
Epoch 8/200
253/253 [==============================] - 28s 110ms/step - d_loss: 0.6888 - g_loss: 0.7562
Epoch 9/200
253/253 [==============================] - 28s 110ms/step - d_loss: 0.6900 - g_loss: 0.7492
Epoch 10/200
253/253 [==============================] - 28s 109m

# X-Ray Non covid model

In [ ]:
# load images
image_size = (128,128)
img_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    covid19_dataset + '/X-ray/Non-COVID/', label_mode=None, image_size=image_size, batch_size=16,crop_to_aspect_ratio=True
)

dataset = img_dataset.map(lambda x: x / 255.0)


Found 5493 files belonging to 1 classes.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
num_channels = 3
num_classes = 1
latent_dim = 256

In [ ]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(128, 128, 3)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.5),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.5),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.5),
        layers.Flatten(),
        layers.Dropout(0.4),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)
discriminator.summary()

# Create the generator.
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(16 * 16 * 128),
        layers.Reshape((16, 16, 128)),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(1024, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, kernel_size=4, padding="same", activation="tanh"),
    ],
    name="generator",
)
generator.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 64)        3136      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 64, 64, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 128)       131200    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 32, 32, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       262272    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 flatten (Flatten)           (None, 32768)           

In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }
    def get_gan():
      return GAN(name='DC_GAN_COVID19_NON_COVID_Xray')



In [ ]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=latent_dim):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        imageFolder = 0
        for i in range(self.num_img):
            img = tf.keras.preprocessing.image.array_to_img(generated_images[i])
            img.save('/content/gdrive/My Drive/Data_Augmented_COVID_Xray_Non_COVID' + '/' + "generated_img_%03d_%d.png" % (epoch, i))


In [ ]:
epochs = 100  # In practice, use ~100 epochs 


gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.RMSprop(learning_rate=0.00001,momentum=0),
    g_optimizer=keras.optimizers.RMSprop(learning_rate=0.00001,momentum=0),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

history = gan.fit(
    dataset, epochs=epochs, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)]
)


model = gan.get_gan
# Save the model
generator.save('/content/gdrive/My Drive/COVID-Data_Augmented_COVID_XrayModelNonCovid/Generator',save_format='tf')
discriminator.save('/content/gdrive/My Drive/COVID-Data_Augmented_COVID_XrayModelNonCovid/Discriminator',save_format='tf')


Epoch 1/100
344/344 [==============================] - 41s 109ms/step - d_loss: 0.6459 - g_loss: 0.9787
Epoch 2/100
344/344 [==============================] - 38s 109ms/step - d_loss: 0.6792 - g_loss: 0.8639
Epoch 3/100
344/344 [==============================] - 38s 109ms/step - d_loss: 0.6997 - g_loss: 0.6929
Epoch 4/100
344/344 [==============================] - 38s 109ms/step - d_loss: 0.6943 - g_loss: 0.7507
Epoch 5/100
344/344 [==============================] - 38s 109ms/step - d_loss: 0.6871 - g_loss: 0.7922
Epoch 6/100
344/344 [==============================] - 38s 110ms/step - d_loss: 0.6799 - g_loss: 0.7965
Epoch 7/100
344/344 [==============================] - 38s 110ms/step - d_loss: 0.6955 - g_loss: 0.7607
Epoch 8/100
344/344 [==============================] - 38s 109ms/step - d_loss: 0.6935 - g_loss: 0.7476
Epoch 9/100
344/344 [==============================] - 38s 109ms/step - d_loss: 0.6914 - g_loss: 0.7570
Epoch 10/100
344/344 [==============================] - 38s 109m

# CT COVID

In [ ]:
# load images
image_size = (128,128)
img_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    covid19_dataset + '/CT/COVID/', label_mode=None, image_size=image_size, batch_size=64,crop_to_aspect_ratio=True
)

dataset = img_dataset.map(lambda x: x / 255.0)


Found 5427 files belonging to 1 classes.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
num_channels = 3
num_classes = 1
latent_dim = 256

In [ ]:
generator_in_channels = latent_dim + num_classes
discriminator_in_channels = num_channels + num_classes
print(generator_in_channels, discriminator_in_channels)

257 4


In [22]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(128, 128, 3)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.5),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.5),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.5),
        layers.Flatten(),
        layers.Dropout(0.4),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)
discriminator.summary()

# Create the generator.
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(16 * 16 * 128),
        layers.Reshape((16, 16, 128)),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(1024, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, kernel_size=4, padding="same", activation="tanh"),
    ],
    name="generator",
)
generator.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 64, 64, 64)        3136      
                                                                 
 leaky_re_lu_32 (LeakyReLU)  (None, 64, 64, 64)        0         
                                                                 
 conv2d_21 (Conv2D)          (None, 32, 32, 128)       131200    
                                                                 
 leaky_re_lu_33 (LeakyReLU)  (None, 32, 32, 128)       0         
                                                                 
 conv2d_22 (Conv2D)          (None, 16, 16, 128)       262272    
                                                                 
 leaky_re_lu_34 (LeakyReLU)  (None, 16, 16, 128)       0         
                                                                 
 flatten_5 (Flatten)         (None, 32768)           

In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }
    def get_gan():
      return GAN(name='DC_GAN_COVID19_COVID_CT')



In [ ]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=latent_dim):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        imageFolder = 0
        for i in range(self.num_img):
            img = tf.keras.preprocessing.image.array_to_img(generated_images[i])
            img.save('/content/gdrive/My Drive/COVID-19 Data Augmented COVID CT' + '/' + "generated_img_%03d_%d.png" % (epoch, i))


In [ ]:
epochs = 100  # In practice, use ~100 epochs 


gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.RMSprop(learning_rate=0.00001,momentum=0),
    g_optimizer=keras.optimizers.RMSprop(learning_rate=0.00001,momentum=0),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

history = gan.fit(
    dataset, epochs=epochs, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)]
)


model = gan.get_gan
# Save the model
generator.save('/content/gdrive/My Drive/COVID-19_Augmented_COVID_CTModel/Generator',save_format='tf')
discriminator.save('/content/gdrive/My Drive/COVID-19_Augmented_COVID_CTModel/Discriminator',save_format='tf')


Epoch 1/100
85/85 [==============================] - 38s 402ms/step - d_loss: 0.6481 - g_loss: 0.8751
Epoch 2/100
85/85 [==============================] - 35s 403ms/step - d_loss: 0.6509 - g_loss: 0.8475
Epoch 3/100
85/85 [==============================] - 35s 403ms/step - d_loss: 0.6708 - g_loss: 0.8067
Epoch 4/100
85/85 [==============================] - 35s 404ms/step - d_loss: 0.6734 - g_loss: 0.7852
Epoch 5/100
85/85 [==============================] - 35s 404ms/step - d_loss: 0.6714 - g_loss: 0.7970
Epoch 6/100
85/85 [==============================] - 35s 404ms/step - d_loss: 0.6750 - g_loss: 0.7838
Epoch 7/100
85/85 [==============================] - 35s 404ms/step - d_loss: 0.6849 - g_loss: 0.7786
Epoch 8/100
85/85 [==============================] - 35s 404ms/step - d_loss: 0.6793 - g_loss: 0.7816
Epoch 9/100
85/85 [==============================] - 35s 404ms/step - d_loss: 0.6639 - g_loss: 0.8048
Epoch 10/100
85/85 [==============================] - 35s 404ms/step - d_loss: 0.6

# CT Non COVID

In [29]:
# load images
image_size = (128,128)
img_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    covid19_dataset + '/CT/Non-COVID/', label_mode=None, image_size=image_size, batch_size=64,crop_to_aspect_ratio=True
)

dataset = img_dataset.map(lambda x: x / 255.0)


Found 2627 files belonging to 1 classes.


In [36]:
num_channels = 3
num_classes = 1
latent_dim = 100

In [37]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(128, 128, 3)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.5),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.5),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.5),
        layers.Flatten(),
        layers.Dropout(0.4),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)
discriminator.summary()

# Create the generator.
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(16 * 16 * 128),
        layers.Reshape((16, 16, 128)),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(1024, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, kernel_size=4, padding="same", activation="tanh"),
    ],
    name="generator",
)
generator.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 64, 64, 64)        3136      
                                                                 
 leaky_re_lu_50 (LeakyReLU)  (None, 64, 64, 64)        0         
                                                                 
 conv2d_33 (Conv2D)          (None, 32, 32, 128)       131200    
                                                                 
 leaky_re_lu_51 (LeakyReLU)  (None, 32, 32, 128)       0         
                                                                 
 conv2d_34 (Conv2D)          (None, 16, 16, 128)       262272    
                                                                 
 leaky_re_lu_52 (LeakyReLU)  (None, 16, 16, 128)       0         
                                                                 
 flatten_8 (Flatten)         (None, 32768)           

In [38]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }
    def get_gan():
      return GAN(name='DC_GAN_COVID19_NON_COVID_CT')



In [39]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=latent_dim):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        imageFolder = 0
        for i in range(self.num_img):
            img = tf.keras.preprocessing.image.array_to_img(generated_images[i])
            img.save('/content/gdrive/My Drive/Data_Augmented_COVID_CT_Non_COVID' + '/' + "generated_img_%03d_%d.png" % (epoch, i))


In [40]:
epochs = 100  # In practice, use ~100 epochs 


gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.RMSprop(learning_rate=0.00001,momentum=0),
    g_optimizer=keras.optimizers.RMSprop(learning_rate=0.00001,momentum=0),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

history = gan.fit(
    dataset, epochs=epochs, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)]
)


model = gan.get_gan
# Save the model
generator.save('/content/gdrive/My Drive/Data_Augmented_COVID_CTModelNonCovid/Generator',save_format='tf')
discriminator.save('/content/gdrive/My Drive/Data_Augmented_COVID_CTModelNonCovid/Discriminator',save_format='tf')


Epoch 1/100
42/42 [==============================] - 20s 399ms/step - d_loss: 0.5517 - g_loss: 0.6872
Epoch 2/100
42/42 [==============================] - 17s 398ms/step - d_loss: 0.4608 - g_loss: 0.6385
Epoch 3/100
42/42 [==============================] - 17s 398ms/step - d_loss: 0.4884 - g_loss: 0.5905
Epoch 4/100
42/42 [==============================] - 17s 398ms/step - d_loss: 0.5075 - g_loss: 0.5908
Epoch 5/100
42/42 [==============================] - 17s 399ms/step - d_loss: 0.4830 - g_loss: 0.6633
Epoch 6/100
42/42 [==============================] - 17s 399ms/step - d_loss: 0.4240 - g_loss: 0.7692
Epoch 7/100
42/42 [==============================] - 17s 399ms/step - d_loss: 0.3804 - g_loss: 0.8234
Epoch 8/100
42/42 [==============================] - 17s 399ms/step - d_loss: 0.3627 - g_loss: 0.8384
Epoch 9/100
42/42 [==============================] - 17s 398ms/step - d_loss: 0.3512 - g_loss: 0.8497
Epoch 10/100
42/42 [==============================] - 17s 399ms/step - d_loss: 0.3